## U shape PHP dynamics with multiple liquid slugs - (under construction)

In [1]:
# using Plots
# pyplot()

In [2]:
# using DifferentialEquations

In [3]:
# using DelimitedFiles

In [4]:
cd(dirname(pwd()))
cd("src")

include("Systems.jl")
include("Tools.jl")

using ..Systems
using ..Tools

## Parameters of the PHP

In [5]:
    ω0 = [sqrt(1.2e004), sqrt(1.2e004)];
    ℘ = [1.2e005, 1.2e005];


    Θ = 0.15;
    θe = 1 + Θ;
    θc = 1 - Θ;
    
    He = 3000.0;
    Hc = 3000.0;
    
    X0 = 0.5;

    γ = 1.4;

In [6]:
    L = 8.0  # total length of the pipe when streched to a 1D pipe
    L2D = 2.0 # the actual length of the bended pipe in the real world
    alpha = pi/2 # inclination angle 

1.5707963267948966

## Pipe definition

In [7]:
tube = Tube(L,L2D,alpha)

Tube(8.0, 2.0, 1.5707963267948966)

## Evaporator and condenser definition

In [8]:
Xe = map(tuple, [0.0; 3.0; 7.0], [1.0; 5.0; 8.0])
evap = Evaporator(He,θe,Xe)

Evaporator(3000.0, 1.15, Tuple{Float64,Float64}[(0.0, 1.0), (3.0, 5.0), (7.0, 8.0)])

In [9]:
Xc = map(tuple, [1.0; 5.0], [3.0; 7.0])
cond = Condenser(Hc,θc,Xc)

Condenser(3000.0, 0.85, Tuple{Float64,Float64}[(1.0, 3.0), (5.0, 7.0)])

## Vapor plug and liquid slug initialization

In [20]:

X0 = [(1.5,3.5),(5.7,7.7)]
dXdt0 = [(0.5,0.5),(0.0,0.0)]
liquids=LiquidSlug(γ,ω0,℘,X0,dXdt0)

LiquidSlug(1.4, [109.54451150103323, 109.54451150103323], [120000.0, 120000.0], Tuple{Float64,Float64}[(1.5, 3.5), (5.7, 7.7)], Tuple{Float64,Float64}[(0.5, 0.5), (0.0, 0.0)])

In [21]:
X0[1][1]

1.5

In [30]:
    P = [1.0,1.0,1.0];
    θ = [1.0,1.0,1.0];
    
    # only good for one liquid slug
    M = zeros(size(θ))
    M[1] = P[1].^(1/γ).*(X0[1][1]);
    M[2] = P[2].^(1/γ).*(X0[2][1]-X0[1][2]);
    M[3] = P[3].^(1/γ).*(L-X0[2][2]);

    vapors=VaporPlug(γ,P)

VaporPlug(1.4, [1.0, 1.0, 1.0])

In [31]:
sys0 = PHPSystem(tube,evap,cond,liquids,vapors)

PHPSystem(Tube(8.0, 2.0, 1.5707963267948966), Evaporator(3000.0, 1.15, Tuple{Float64,Float64}[(0.0, 1.0), (3.0, 5.0), (7.0, 8.0)]), Condenser(3000.0, 0.85, Tuple{Float64,Float64}[(1.0, 3.0), (5.0, 7.0)]), LiquidSlug(1.4, [109.54451150103323, 109.54451150103323], [120000.0, 120000.0], Tuple{Float64,Float64}[(1.5, 3.5), (5.7, 7.7)], Tuple{Float64,Float64}[(0.5, 0.5), (0.0, 0.0)]), VaporPlug(1.4, [1.0, 1.0, 1.0]))

In [32]:
typeof(sys0)

PHPSystem

In [33]:
getheight(sys0)

2-element Array{Tuple{Float64,Float64},1}:
 (0.5, 1.5)                              
 (0.2999999999999998, 1.7000000000000002)

 ## $\textbf{U}=\left(\begin{array}{c}  X_{p1}   \\  X_{p2} \\  X_{p3} \\  X_{p4} \\ \frac{\mathrm{d} X_{p1}}{\mathrm{d} \tau}  \\ \frac{\mathrm{d} X_{p2}}{\mathrm{d} \tau}   \\ \frac{\mathrm{d} X_{p3}}{\mathrm{d} \tau}   \\ \frac{\mathrm{d} X_{p4}}{\mathrm{d} \tau} \\ M_{1} \\ M_{2} \\ M_{3} \end{array}\right)$.

In [34]:
function solve!(du,u,p,t)
    
    sys0 = p
    
    γ = sys0.liquidslug.γ
    ω0 = sys0.liquidslug.ω0
    ℘ = sys0.liquidslug.℘
    Lvaporplug = XptoLvaporplug(sys0,Xp)
    
    
    (Xp,dXdt0,M)=vectoXM(u)
    
    
    # get P from M and γ
    P = (M./Lvaporplug).^(γ) 
    
    
    # get θ from P and γ
    θ = P.^((γ-1)/γ)

    
    du[1] = u[2]
    du[2] = -32*u[2] - (ω0^2)*u[1] + ζ*(P[1]-P[2])
    u[1] > 0 ? du[3] = -Hc*u[1]*(θ[1] - θc) : du[3] = He*(1 + u[1])*(θe - θ[1]) 
    u[1] > 0 ? du[4] = He*(1-u[1])*(θe - θ[2]) : du[4] = Hc*u[1]*(θ[2] - θc)
    
    return du
    
end

solve! (generic function with 1 method)

In [35]:
function systovec(sys0,Xp,dXdt,M)
    if (length(Xp) == length(dXdt)) && (2 * length(Xp) == length(M) + 1)
        
        u=zeros(3*2*length(Xp)-1)
        
        for i = 1:length(Xp)
            
            # input Xp
            u[2*i-1] = Xp[i][1]
            u[2*i] = Xp[i][end]
            
            # input dXdt
            u[2*length(Xp) + 2*i-1] = dXdt[i][1]
            u[2*length(Xp) + 2*i] = dXdt[i][end]           
        end
        
        for i = 1:length(M)            
            # input M
            u[4*length(Xp) + i] = M[i]
        end
    else
        println("the lengthes of X and dXdt and M do not match!")
    end
    
    return u
            
end

systovec (generic function with 1 method)

In [74]:
function vectoXM(sys0,u)
    
    maxindex = Integer( (length(u) + 1)/3/2 )
    
    Xp = map(tuple, zeros(maxindex), zeros(maxindex))
    dXdt = map(tuple, zeros(maxindex), zeros(maxindex))
    M = zeros(2*maxindex-1)
    
    for i = 1:maxindex
        
        # input Xp
        Xp[i] = (u[2*i-1],u[2*i])
        
        # input dXdt
        dXdt[i] = (u[2*maxindex + 2*i-1],u[2*maxindex + 2*i])
    end
    
    for i = 1:(2*maxindex-1)
        
        # input M
        M[i] = u[4*maxindex + i]

    end
    
    return Xp,dXdt,M
            
end

vectoXM (generic function with 1 method)

In [81]:
function XptoLvaporplug(sys0,Xp)
    
    L = sys0.tube.L
    
    maxindex = length(Xp) + 1
    Lvaporplug = zeros(maxindex)

    Lvaporplug[1] = Xp[1][1]-0.0
    Lvaporplug[end] = L-Xp[end][end]
    
    if maxindex > 2
        for i = 2:maxindex-1
        
            Lvaporplug[i] = Xp[i][1] - Xp[i-1][end]
            
        end    
    end
  
    return Lvaporplug
            
end

XptoLvaporplug (generic function with 1 method)

In [76]:
u=systovec(X0,dXdt0,M)

11-element Array{Float64,1}:
 1.5               
 3.5               
 5.7               
 7.7               
 0.5               
 0.5               
 0.0               
 0.0               
 1.5               
 2.2               
 0.2999999999999998

In [77]:
(Xp,dXdt0,M)=vectoXM(u)

(Tuple{Float64,Float64}[(1.5, 3.5), (5.7, 7.7)], Tuple{Float64,Float64}[(0.5, 0.5), (0.0, 0.0)], [1.5, 2.2, 0.2999999999999998])

In [84]:
Lvaporplug=XptoLvaporplug(sys0,X0)

3-element Array{Float64,1}:
 1.5               
 2.2               
 0.2999999999999998

In [85]:
P = (M./Lvaporplug).^(γ) 

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

In [86]:
 θ = P.^((γ-1)/γ)

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

In [87]:
du=zeros(size(u))

11-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0